# Библиотека Emma

## Тексификация

В этой тетрадке демонстрируются возможности начальной версии библиотеки Emma для тексификации текста. Работу алгоритмов еще предстоит улучшить.

**Полная тексификация** — это преобразование исходного текста, содержащего математические сущности, выраженные в смеси полусловесных формулировок и/или различных математических языков разметки, в текст, приведенный в соответствие с правилами системы компьютерной вёрстки LaTeX для русского языка, обладающий большим уровнем восприятия, в котором выделены математические сущности.

### Импорты и установка

Сделаем необходимые импорты и установим библиотеку Emma.

In [1]:
import sys
from IPython.core.display import display, HTML


render = lambda x: display(HTML(x))

In [2]:
!git clone https://github.com/basic-go-ahead/emma
!pip install -q -r ./emma/requirements.txt

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Cloning into 'emma'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 19 (delta 4), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (19/19), 11.26 KiB | 1.41 MiB/s, done.
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [3]:
sys.path.append('./emma')

from emma import Texificator

### Создание тексификатора

Теперь мы можем создать тексификатор:

In [4]:
texificator = Texificator()

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/303 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/756 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

Тексификатор инкапсулирует модель [`basic-go/rut5-base-texificator`](https://huggingface.co/basic-go/rut5-base-texificator) и упрощает взаомодействие с ней.

### Обработка результатов транскрибации

Непосредственно сама тексификация осуществляется методом `doit`, который принимает строку или набор строк:

In [5]:
texified = texificator.doit("шинус два икс минус чосинус игрек равно ноль")

render(texified)

In [6]:
items = [
    'икс квадрат плюс игрек минус двадцать два зет',
    'а минус б квадрат умножить на ц',
    'дробь числитель икс знаменатель десять',
    'дробь числитель икс квадрат минус игрек знаменатель икс плюс десять'
]

for texified in texificator.doit(items):
    render(texified)

---

Как видно, тексификатор не плохо справляется с текстом, который можно назвать транскрибацией того, как бы мы произнесли соответствующее математическое выражение.

### Обработка микс-представлений математических выражений

Также можно тексифицировать комбинацию словесных формулировок, чисел и математических выражений:

In [7]:
items = [
    '100 вторых',
    'a * b - c^2 + (x + y) минус десять',
    '1 - синус икс + косинус квадрат x больше или равно -1'
]

for texified in texificator.doit(items):
    render(texified)

### Обработка выражений в тексте

Наконец, тексификатор способен отличать обычный текст от математических выражений:

In [8]:
items = [
    'Скорость составляет 10 км в час.',
    'Треугольник АБЦ - равнобедренный',
    'Скорость космического аппарата зависит от t по формуле v(t) = 10 t**2 - 5',
    'Ребята потратили в прошлом месяце 10000 рублей'
]

for texified in texificator.doit(items):
    render(texified)

---

Как видно, тексификатор может сокращать фрагменты текста до известных обозначений, вставлять неразрывные пробелы необходимого типа в зависимости от контекста, выделять длинное тире, характерное для русской типографии, а также, как побочное действие, в некоторых случаях исправлять пунктуационные ошибки.

Продолжаются дальнейшие работы в этом направлении.